# 01 — API Exploration

Explore all retrieval APIs and medical NLP modules built in Stage 1.

**What we're testing:**
1. Medical NER (scispaCy) — entity extraction from health claims
2. PICO Extraction — rule-based structuring of claims
3. MeSH Mapping — mapping entities to PubMed vocabulary
4. PubMed API — search and fetch biomedical articles
5. Semantic Scholar API — paper search with citation data
6. ClinicalTrials.gov API — search registered trials
7. OpenFDA / RxNorm — drug labels and interactions
8. Cochrane — systematic review search

In [1]:
import sys
sys.path.insert(0, '..')

# Suppress warnings from scispaCy
import warnings
warnings.filterwarnings('ignore')

## Test Claims

We'll use these claims throughout the notebook.

In [2]:
CLAIMS = [
    "Intermittent fasting reverses Type 2 diabetes",
    "Vitamin D prevents COVID-19 infection",
    "Metformin reduces cancer risk in diabetic patients",
    "Turmeric is as effective as ibuprofen for arthritis pain",
    "Drinking 8 glasses of water a day prevents kidney stones",
]

---
## 1. Medical NER (scispaCy)

Extract drugs, conditions, genes, and other medical entities from claims.

In [3]:
from src.medical_nlp.medical_ner import extract_entities

for claim in CLAIMS:
    entities = extract_entities(claim)
    print(f"Claim: {claim}")
    print(f"  Drugs:      {entities.drugs}")
    print(f"  Conditions: {entities.conditions}")
    print(f"  Genes:      {entities.genes}")
    print(f"  Procedures: {entities.procedures}")
    print(f"  Raw:        {[(e['text'], e['label']) for e in entities.raw_entities]}")
    print()

Claim: Intermittent fasting reverses Type 2 diabetes
  Drugs:      []
  Conditions: ['Intermittent fasting', 'Type 2 diabetes']
  Genes:      []
  Procedures: []
  Raw:        [('Intermittent fasting', 'ENTITY'), ('Type 2 diabetes', 'ENTITY')]

Claim: Vitamin D prevents COVID-19 infection
  Drugs:      []
  Conditions: ['Vitamin D', 'COVID-19', 'infection']
  Genes:      []
  Procedures: []
  Raw:        [('Vitamin D', 'ENTITY'), ('COVID-19', 'ENTITY'), ('infection', 'ENTITY')]

Claim: Metformin reduces cancer risk in diabetic patients
  Drugs:      ['Metformin']
  Conditions: ['cancer risk', 'diabetic', 'patients']
  Genes:      []
  Procedures: []
  Raw:        [('Metformin', 'ENTITY'), ('cancer risk', 'ENTITY'), ('diabetic', 'ENTITY'), ('patients', 'ENTITY')]

Claim: Turmeric is as effective as ibuprofen for arthritis pain
  Drugs:      ['ibuprofen']
  Conditions: ['Turmeric', 'arthritis pain']
  Genes:      []
  Procedures: []
  Raw:        [('Turmeric', 'ENTITY'), ('ibuprofen', 'E

---
## 2. PICO Extraction — Rule-Based vs LLM

Compare both extraction methods side-by-side.

In [4]:
from src.medical_nlp.pico_extractor import extract_pico_rule_based
from src.medical_nlp.medical_ner import extract_entities

print("=" * 80)
print("RULE-BASED PICO (no LLM, free)")
print("=" * 80)

for claim in CLAIMS:
    entities = extract_entities(claim)
    pico = extract_pico_rule_based(claim, entities)
    print(f"\nClaim: {claim}")
    print(f"  P (Population):   {pico.population}")
    print(f"  I (Intervention): {pico.intervention}")
    print(f"  C (Comparison):   {pico.comparison}")
    print(f"  O (Outcome):      {pico.outcome}")

RULE-BASED PICO (no LLM, free)

Claim: Intermittent fasting reverses Type 2 diabetes
  P (Population):   None
  I (Intervention): Intermittent fasting
  C (Comparison):   None
  O (Outcome):      Type 2 diabetes

Claim: Vitamin D prevents COVID-19 infection
  P (Population):   None
  I (Intervention): Vitamin D
  C (Comparison):   None
  O (Outcome):      COVID-19 infection

Claim: Metformin reduces cancer risk in diabetic patients
  P (Population):   diabetic patients
  I (Intervention): Metformin
  C (Comparison):   None
  O (Outcome):      cancer risk

Claim: Turmeric is as effective as ibuprofen for arthritis pain
  P (Population):   None
  I (Intervention): Turmeric
  C (Comparison):   ibuprofen
  O (Outcome):      arthritis pain

Claim: Drinking 8 glasses of water a day prevents kidney stones
  P (Population):   None
  I (Intervention): Drinking 8 glasses of water a day
  C (Comparison):   None
  O (Outcome):      kidney stones


In [6]:
from src.medical_nlp.pico_extractor import extract_pico_with_llm

print("=" * 80)
print("LLM-BASED PICO (Claude Sonnet)")
print("=" * 80)

for claim in CLAIMS:
    entities = extract_entities(claim)
    pico = extract_pico_with_llm(claim, entities)
    print(f"\nClaim: {claim}")
    print(f"  P (Population):   {pico.population}")
    print(f"  I (Intervention): {pico.intervention}")
    print(f"  C (Comparison):   {pico.comparison}")
    print(f"  O (Outcome):      {pico.outcome}")

LLM-BASED PICO (Claude Sonnet)

Claim: Intermittent fasting reverses Type 2 diabetes
  P (Population):   patients with Type 2 diabetes
  I (Intervention): intermittent fasting
  C (Comparison):   no treatment
  O (Outcome):      reversal of Type 2 diabetes

Claim: Vitamin D prevents COVID-19 infection
  P (Population):   general population
  I (Intervention): Vitamin D
  C (Comparison):   no Vitamin D
  O (Outcome):      prevention of COVID-19 infection

Claim: Metformin reduces cancer risk in diabetic patients
  P (Population):   diabetic patients
  I (Intervention): metformin
  C (Comparison):   no treatment
  O (Outcome):      reduced cancer risk

Claim: Turmeric is as effective as ibuprofen for arthritis pain
  P (Population):   patients with arthritis pain
  I (Intervention): turmeric
  C (Comparison):   ibuprofen
  O (Outcome):      arthritis pain relief

Claim: Drinking 8 glasses of water a day prevents kidney stones
  P (Population):   General population
  I (Intervention): Dri

---
## 3. MeSH Mapping

Map extracted entities to MeSH descriptors for better PubMed search queries.

In [ ]:
import pandas as pd

# Side-by-side comparison
rows = []
for claim in CLAIMS:
    entities = extract_entities(claim)
    rule = extract_pico_rule_based(claim, entities)
    llm = extract_pico_with_llm(claim, entities)
    rows.append({
        "Claim": claim[:45] + "..." if len(claim) > 45 else claim,
        "Rule P": rule.population or "-",
        "LLM P": llm.population or "-",
        "Rule I": rule.intervention or "-",
        "LLM I": llm.intervention or "-",
        "Rule C": rule.comparison or "-",
        "LLM C": llm.comparison or "-",
        "Rule O": rule.outcome or "-",
        "LLM O": llm.outcome or "-",
    })

df = pd.DataFrame(rows)
print("Side-by-side: Rule-Based vs LLM\n")
print(df.to_string(index=False))

In [5]:
from src.medical_nlp.mesh_mapper import map_entities_to_mesh

test_terms = ["Type 2 diabetes", "intermittent fasting", "metformin", "COVID-19", "turmeric"]

mesh_results = map_entities_to_mesh(test_terms)

for term, mesh in mesh_results.items():
    if mesh:
        print(f"{term}")
        print(f"  MeSH Name: {mesh.name}")
        print(f"  UID:       {mesh.uid}")
        print(f"  Trees:     {mesh.tree_numbers}")
    else:
        print(f"{term} -> No MeSH match")
    print()

Type 2 diabetes
  MeSH Name: Diabetes Mellitus, Type 2
  UID:       68003924
  Trees:     []

intermittent fasting
  MeSH Name: Intermittent Fasting
  UID:       2103248
  Trees:     []

metformin
  MeSH Name: Phenformin
  UID:       68010629
  Trees:     []

COVID-19
  MeSH Name: Masks
  UID:       68008397
  Trees:     []

turmeric
  MeSH Name: Curcuma
  UID:       68030024
  Trees:     []



---
## 4. PubMed API

Search PubMed with free-text queries and PICO-structured queries. Fetch article details including MeSH terms and publication types.

In [7]:
from src.retrieval.pubmed_client import search_and_fetch, build_query_from_pico

# Simple search
articles = search_and_fetch("intermittent fasting type 2 diabetes", max_results=5)

print(f"PubMed returned {len(articles)} articles:\n")
for a in articles:
    print(f"[PMID:{a.pmid}] {a.title}")
    print(f"  Journal:  {a.journal}")
    print(f"  Date:     {a.pub_date}")
    print(f"  MeSH:     {a.mesh_terms[:5]}")
    print(f"  Types:    {a.publication_types}")
    print(f"  PMC:      {a.pmc_id or 'N/A'}")
    print(f"  URL:      {a.url}")
    print(f"  Abstract: {a.abstract[:200]}..." if len(a.abstract) > 200 else f"  Abstract: {a.abstract}")
    print()

PubMed returned 5 articles:

[PMID:38956175] The effects of intermittent fasting on body composition and cardiometabolic health in adults with prediabetes or type 2 diabetes: A systematic review and meta-analysis.
  Journal:  Diabetes, obesity & metabolism
  Date:     2024 Sep
  MeSH:     ['Humans', 'Diabetes Mellitus, Type 2', 'Prediabetic State', 'Fasting', 'Body Composition']
  Types:    ['Journal Article', 'Meta-Analysis', 'Systematic Review']
  PMC:      N/A
  URL:      https://pubmed.ncbi.nlm.nih.gov/38956175/
  Abstract: To perform a meta-analysis to investigate the effects of intermittent fasting (IF), as compared with either a control diet (CON) and/or calorie restriction (CR), on body composition and cardiometaboli...

[PMID:36508320] Efficacy and Safety of Intermittent Fasting in People With Insulin-Treated Type 2 Diabetes (INTERFAST-2)-A Randomized Controlled Trial.
  Journal:  Diabetes care
  Date:     2023 Feb
  MeSH:     ['Humans', 'Blood Glucose', 'Blood Glucose Self-Mo

In [ ]:
# PICO-structured search
pico_query = build_query_from_pico(
    population="Type 2 diabetes",
    intervention="intermittent fasting",
    outcome="glycemic control OR HbA1c",
)
print(f"PICO query: {pico_query}\n")

pico_articles = search_and_fetch(pico_query, max_results=5)
print(f"Returned {len(pico_articles)} articles:\n")
for a in pico_articles:
    print(f"  [{a.pmid}] {a.title[:100]}")

---
## 5. Semantic Scholar API

Search with citation counts, TLDRs, and open-access PDF links.

In [8]:
from src.retrieval.semantic_scholar import search as s2_search, get_paper

papers = s2_search("intermittent fasting type 2 diabetes", max_results=5)

print(f"Semantic Scholar returned {len(papers)} papers:\n")
for p in papers:
    print(f"[{p.year}] {p.title}")
    print(f"  Citations:     {p.citation_count} (influential: {p.influential_citation_count})")
    print(f"  Venue:         {p.venue}")
    print(f"  PMID:          {p.pmid or 'N/A'}")
    print(f"  DOI:           {p.doi or 'N/A'}")
    print(f"  Open Access:   {p.is_open_access}")
    print(f"  PDF:           {p.open_access_pdf_url or 'N/A'}")
    print(f"  TLDR:          {p.tldr or 'N/A'}")
    print(f"  Fields:        {p.fields_of_study}")
    print()

Semantic Scholar returned 5 papers:

[2025] Intermittent fasting for glycemic control in patients with type 2 diabetes: a meta-analysis of randomized controlled trials.
  Citations:     5 (influential: 0)
  Venue:         Nutrición Hospitalaria
  PMID:          40008664
  DOI:           10.20960/nh.05521
  Open Access:   False
  PDF:           N/A
  TLDR:          IF can reduce body weight, reduce waist circumference, maintain stable blood pressure, and reduce low-density lipoprotein and total cholesterol levels, and is considered safe and feasible to implement.
  Fields:        ['Medicine']

[2025] Comparison of the Effect of Intermittent Fasting with Mediterranean Diet on Glycemic, Lipid, and Anthropometric Indices in Type 2 Diabetes: A Review of Randomized Controlled Trials.
  Citations:     2 (influential: 0)
  Venue:         Current hypertension reviews
  PMID:          40231524
  DOI:           10.2174/0115734021351456250326051146
  Open Access:   False
  PDF:           N/A
  TLD

In [ ]:
# Fetch a specific paper by PMID
if papers and papers[0].pmid:
    paper = get_paper(f"PMID:{papers[0].pmid}")
    if paper:
        print(f"Fetched by PMID: {paper.title}")
        print(f"  Abstract: {paper.abstract[:300]}..." if paper.abstract and len(paper.abstract) > 300 else f"  Abstract: {paper.abstract}")
else:
    print("No paper with PMID to test")

---
## 6. ClinicalTrials.gov API

Search registered clinical trials. Filter by status, phase, and study type.

In [9]:
from src.retrieval.clinical_trials import search as ct_search, search_by_condition_and_intervention

trials = ct_search("intermittent fasting diabetes", max_results=5)

print(f"ClinicalTrials.gov returned {len(trials)} trials:\n")
for t in trials:
    print(f"[{t.nct_id}] {t.title}")
    print(f"  Status:       {t.status}")
    print(f"  Phase:        {t.phase or 'N/A'}")
    print(f"  Type:         {t.study_type}")
    print(f"  Enrollment:   {t.enrollment or 'N/A'}")
    print(f"  Conditions:   {t.conditions}")
    print(f"  Interventions:{t.interventions}")
    print(f"  Outcomes:     {t.primary_outcomes[:2]}")
    print(f"  Sponsor:      {t.sponsor}")
    print(f"  Dates:        {t.start_date} → {t.completion_date}")
    print(f"  Has results:  {t.has_results}")
    print(f"  URL:          {t.url}")
    print()

ClinicalTrials.gov returned 5 trials:

[NCT02450097] Metabolic and Hormonal Effects of 5:2 Intermittent Fasting in Patients With Type 2 Diabetes and Subjects With Adiposity
  Status:       COMPLETED
  Phase:        NA
  Type:         INTERVENTIONAL
  Enrollment:   100
  Conditions:   ['Obesity', 'Diabetes Type 2']
  Interventions:['OTHER: Caloric restriction']
  Outcomes:     ['Change from baseline in fasting serum insulin in 6 months.']
  Sponsor:      Karolinska University Hospital
  Dates:        2013-12 → 2017-12
  Has results:  False
  URL:          https://clinicaltrials.gov/study/NCT02450097

[NCT06911918] The Effect of Time Restricted Eating on Glycemic and Proinflammatory Biomarkers Among Prediabetic Obese Jordanian Adults Aged 18-40 Years Old: A Randomized Control Study
  Status:       NOT_YET_RECRUITING
  Phase:        NA
  Type:         INTERVENTIONAL
  Enrollment:   120
  Conditions:   ['Insulin Resistance', 'Inflammation Biomarkers', 'Time Restricted Eating', 'Caloric Res

In [10]:
# PICO-style search: condition + intervention, completed only
completed = search_by_condition_and_intervention(
    condition="Type 2 diabetes",
    intervention="intermittent fasting",
    max_results=5,
    completed_only=True,
)

print(f"Completed trials: {len(completed)}\n")
for t in completed:
    print(f"  [{t.nct_id}] {t.title[:80]}")
    print(f"    Enrollment: {t.enrollment}, Results: {t.has_results}")

Completed trials: 5

  [NCT02450097] Metabolic and Hormonal Effects of 5:2 Intermittent Fasting in Patients With Type
    Enrollment: 100, Results: False
  [NCT02498002] Impacts of Intermittent Fasting in Adults on Energy Balance, Body Composition, P
    Enrollment: 54, Results: False
  [NCT04348019] Mealtime Matters: An 8-wk Randomized-Controlled Trial to Examine the Effects of 
    Enrollment: 29, Results: False
  [NCT05375695] Feasibility of Time-Restricted Eating in the Treatment of Type 2 Diabetes - the 
    Enrollment: 20, Results: False
  [NCT06478173] The Effect of Complex Exercise Training on General Health Status in Patients wit
    Enrollment: 126, Results: False


---
## 7. OpenFDA + RxNorm (Drug Info)

Look up drug labels (indications, warnings, interactions) and check drug-drug interactions.

In [11]:
from src.retrieval.drugbank_client import search_drug_label, get_interactions

# Drug label lookup
drugs = search_drug_label("metformin", max_results=1)

if drugs:
    d = drugs[0]
    print(f"Drug: {d.generic_name}")
    print(f"  Brand names:    {d.brand_names[:5]}")
    print(f"  RxCUI:          {d.rxcui}")
    print(f"  Indications:    {d.indications[:300]}..." if len(d.indications) > 300 else f"  Indications:    {d.indications}")
    print(f"  Warnings:       {d.warnings[:300]}..." if len(d.warnings) > 300 else f"  Warnings:       {d.warnings}")
    print(f"  Contraindic.:   {d.contraindications[:300]}..." if len(d.contraindications) > 300 else f"  Contraindic.:   {d.contraindications}")
    print(f"  Interactions:   {d.drug_interactions[:300]}..." if len(d.drug_interactions) > 300 else f"  Interactions:   {d.drug_interactions}")
else:
    print("No drug info found")

Drug: SITAGLIPTIN AND METFORMIN HYDROCHLORIDE
  Brand names:    ['ZITUVIMET']
  RxCUI:          861769
  Indications:    1 INDICATIONS AND USAGE ZITUVIMET is a combination of sitagliptin, a dipeptidyl peptidase-4 (DPP-4) inhibitor, and metformin hydrochloride (HCl), a biguanide, indicated as an adjunct to diet and exercise to improve glycemic control in adults with type 2 diabetes mellitus. ( 1 ) Limitations of Use: Z...
  Warnings:       
  Contraindic.:   4 CONTRAINDICATIONS Severe renal impairment: (eGFR below 30 mL/min/1.73 m 2 ) ( 4) Metabolic acidosis, including diabetic ketoacidosis. ( 4 ) History of a serious hypersensitivity reaction to ZITUVIMET, sitagliptin, or metformin, such as anaphylaxis or angioedema. (4) ZITUVIMET is contraindicated in...
  Interactions:   7 DRUG INTERACTIONS Table 4 presents clinically significant drug interactions with ZITUVIMET: Table 4: Clinically Significant Drug Interactions with ZITUVIMET Carbonic Anhydrase Inhibitors Clinical Impact: Carbonic a

In [12]:
# Drug-drug interactions via RxNorm
interactions = get_interactions("metformin")

print(f"Found {len(interactions)} interactions for metformin:\n")
for ix in interactions[:10]:  # Show first 10
    print(f"  {ix.drug_a} ↔ {ix.drug_b}")
    print(f"    Severity: {ix.severity}")
    print(f"    {ix.description[:150]}")
    print()

RxNorm interaction lookup failed: 404 Client Error: Not Found for url: https://rxnav.nlm.nih.gov/REST/interaction/interaction.json?rxcui=6809


Found 0 interactions for metformin:



---
## 8. Cochrane Systematic Reviews

Search for Cochrane reviews (gold standard evidence synthesis) via PubMed + Semantic Scholar.

In [13]:
from src.retrieval.cochrane_client import search_cochrane

reviews = search_cochrane(
    query="intermittent fasting diabetes",
    max_results=5,
    population="Type 2 diabetes",
    intervention="intermittent fasting",
)

print(f"Cochrane returned {len(reviews)} reviews:\n")
for r in reviews:
    print(f"[{r.year}] {r.title}")
    print(f"  DOI:     {r.doi}")
    print(f"  Authors: {', '.join(r.authors[:3])}{'...' if len(r.authors) > 3 else ''}")
    print(f"  URL:     {r.url}")
    if r.abstract:
        print(f"  Abstract: {r.abstract[:200]}...")
    print()

Cochrane returned 5 reviews:

[2014] Techniques of monitoring blood glucose during pregnancy for women with pre-existing diabetes.
  DOI:     10.1002/14651858.CD009613.pub2
  Authors: Moy Foong Ming, Ray Amita, Buckley Brian S
  URL:     https://doi.org/10.1002/14651858.CD009613.pub2
  Abstract: Self-monitoring of blood glucose is recommended as a key component of the management plan for diabetes therapy during pregnancy. No existing systematic reviews consider the benefits/effectiveness of v...

[2017] Techniques of monitoring blood glucose during pregnancy for women with pre-existing diabetes.
  DOI:     10.1002/14651858.CD009613.pub3
  Authors: Moy Foong Ming, Ray Amita, Buckley Brian S...
  URL:     https://doi.org/10.1002/14651858.CD009613.pub3
  Abstract: Self-monitoring of blood glucose (SMBG) is recommended as a key component of the management plan for diabetes therapy during pregnancy. No existing systematic reviews consider the benefits/effectivene...

[2025] Continuous gluco

---
## 9. End-to-End: Claim → Entities → PICO → Multi-Source Search

Combine everything: take a raw claim, extract entities and PICO, then search all sources.

In [14]:
import time
from src.medical_nlp.medical_ner import extract_entities
from src.medical_nlp.pico_extractor import extract_pico_rule_based
from src.retrieval.pubmed_client import search_and_fetch, build_query_from_pico
from src.retrieval.semantic_scholar import search as s2_search
from src.retrieval.clinical_trials import search_by_condition_and_intervention
from src.retrieval.cochrane_client import search_cochrane

claim = "Vitamin D prevents COVID-19 infection"
print(f"=== Claim: {claim} ===\n")

# Step 1: NER
entities = extract_entities(claim)
print(f"Entities: {entities.all_entities()}")

# Step 2: PICO
pico = extract_pico_rule_based(claim, entities)
print(f"PICO: P={pico.population}, I={pico.intervention}, C={pico.comparison}, O={pico.outcome}\n")

# Step 3: Search all sources
query_terms = ' '.join(entities.all_entities()) or claim

print("--- PubMed ---")
pm_articles = search_and_fetch(query_terms, max_results=3)
for a in pm_articles:
    print(f"  [{a.pmid}] {a.title[:80]}")

print("\n--- Semantic Scholar ---")
time.sleep(1)  # rate limit
s2_papers = s2_search(query_terms, max_results=3)
for p in s2_papers:
    print(f"  [{p.year}] {p.title[:80]} (cited: {p.citation_count})")

print("\n--- ClinicalTrials.gov ---")
ct_trials = search_by_condition_and_intervention(
    condition=pico.population or claim,
    intervention=pico.intervention or "",
    max_results=3,
)
for t in ct_trials:
    print(f"  [{t.nct_id}] {t.title[:80]} ({t.status})")

print("\n--- Cochrane ---")
c_reviews = search_cochrane(query_terms, max_results=3)
for r in c_reviews:
    print(f"  [{r.year}] {r.title[:80]}")

print(f"\n=== Total evidence found: {len(pm_articles)} articles + {len(s2_papers)} papers + {len(ct_trials)} trials + {len(c_reviews)} reviews ===")

=== Claim: Vitamin D prevents COVID-19 infection ===

Entities: ['Vitamin D', 'COVID-19', 'infection']
PICO: P=Vitamin D, I=None, C=None, O=infection

--- PubMed ---
  [38474807] Preventive Vitamin D Supplementation and Risk for COVID-19 Infection: A Systemat
  [36308699] Vitamin D-A prominent immunomodulator to prevent COVID-19 infection.
  [32912355] Vitamin D deficiency in schizophrenia implications for COVID-19 infection.

--- Semantic Scholar ---
  [2020] Low plasma 25(OH) vitamin D level is associated with increased risk of COVID‐19  (cited: 401)
  [2020] Vitamin D sufficiency, a serum 25-hydroxyvitamin D at least 30 ng/mL reduced ris (cited: 245)
  [2020] Role of vitamin D in preventing of COVID-19 infection, progression and severity (cited: 427)

--- ClinicalTrials.gov ---
  [NCT01490333] Sunweavers: Supporting Native American Women's Vitamin D Research (COMPLETED)
  [NCT07143552] A Prospective Case-Control Study of Vitamin D Status, Bone Metabolic Markers, an (RECRUITING)
  [N

Cochrane/S2 search failed: 429 Client Error:  for url: https://api.semanticscholar.org/graph/v1/paper/search?query=Vitamin+D+COVID-19+infection+Cochrane+systematic+review&limit=3&fields=paperId%2Ctitle%2Cabstract%2Cyear%2Cauthors%2Cvenue%2CexternalIds&venue=The+Cochrane+database+of+systematic+reviews


  [2023] Vitamin D for the management of asthma.
  [2024] Vitamin D for the management of chronic obstructive pulmonary disease.
  [2021] Vitamin D supplementation for the treatment of COVID-19: a living systematic rev

=== Total evidence found: 3 articles + 3 papers + 3 trials + 3 reviews ===
